# DSAI HW1 Peak Load Forecasting
請根據台電歷史資料，預測未來七天的"電力尖峰負載"(MW)

In [434]:
import numpy as np
import pandas as pd
from keras import backend as K
from keras.models import Sequential 
from keras.layers import Dense

## Data Preprocessing

Read data

In [421]:
def readfile(path):
    file = open(path, "r+")
    f = file.readlines()
    return_tp_list = []
    test_list = []
    for i in f[1:366]:
        #print(i)
        day_list = i[:-1].split(",")
        temp_list = []
        for d in day_list[:2]:
            temp_list.append(d)
        for d in day_list[3:5]:
            temp_list.append(d)
        test_list.append(day_list[2])
        #print(temp_list)
        #print(len(temp_list))
        return_tp_list.append(temp_list)
    
    return return_tp_list, test_list

In [422]:
#tp_2017  = pandas.read_csv('data/taipower_2017.csv')[:365]
#tp_2018  = pandas.read_csv('data/taipower_2018.csv')[:365]
tp_2017, target_2017_list = readfile('data/taipower_2017.csv')
tp_2018, target_2018_list = readfile('data/taipower_2018.csv')

#print(tp_2018)

Parser data to get the same week in last year and last week 

In [423]:
def sliceData(tp_data_2017, tp_data_2018):
    #index = 1
    con_list = []
    data_2017_list = []
    for t in range(len(tp_data_2017[:-1])):
        if (t+1) % 7 is not 0:
            con_list += tp_data_2017[t][1:]
        else:
            #if t is 0:
            con_list += tp_data_2017[t][1:]
            data_2017_list.append(con_list)
            con_list = []
    
    con_list = []
    data_2018_list = []
    for t in range(len(tp_data_2018[:-1])):
        if (t+1) % 7 is not 0:
            con_list += tp_data_2018[t][1:]
            #print(con_list)
        else:
            #if t is 0:
            con_list += tp_data_2018[t][1:]    
            data_2018_list.append(con_list)
            con_list = []
    
    #print("2017", data_2017_list[0])
    #print("2018", data_2018_list[0])
    train_list = []
    temp_list =[]
    for i in range(len(data_2017_list)):
        if i == 0:
            temp_list = data_2017_list[i] + data_2017_list[-1]
            #print("i = 0", temp_list)
        else:
            temp_list = data_2017_list[i] + data_2017_list[i]
            #print("i != 0", temp_list)
        train_list.append(temp_list)
    
    return train_list

In [424]:
train_X = sliceData(tp_2017, tp_2018)
print(train_X[0])
print(len(train_X[0]))

['24957', '2071', '9.05', '26035', '1552', '6.34', '29152', '1021', '3.63', '29757', '1272', '4.47', '30065', '1729', '6.1', '29154', '1152', '4.12', '27814', '2214', '8.65', '27219', '2959', '12.2', '30394', '2455', '8.79', '30723', '2262', '7.95', '30500', '2478', '8.84', '30583', '2354', '8.34', '30204', '2835', '10.36', '28107', '3389', '13.71']
42


In [425]:
def getTrainY(target_list):
    train_y_list = []
    con_list = []
    for i in range(len(target_list[:-1])):
        if (i+1) % 7 is not 0:
            con_list.append(target_list[i])
        else:
            con_list.append(target_list[i])
            train_y_list.append(con_list)
            con_list = []
    return train_y_list

In [426]:
train_Y = getTrainY(target_2018_list)
#print(train_Y)
np_train_X = np.array(train_X)
np_train_Y = np.array(train_Y)

# np_train_X = np_train_X[:,:,np.newaxis]
# np_train_Y = np_train_X[:,:,np.newaxis]

#np_train_X  = np.reshape(np_train_X , (1, np_train_X.shape[0], np_train_X.shape[1]))
#np_train_Y  = np.reshape(np_train_Y , (1, np_train_Y.shape[0], np_train_Y.shape[1]))
#n = np_train_X.reshape(52, 1, 42)
#print(n.shape)
#print(np_train_X.shape)

In [427]:
def bulidModel(x_shape):
    model = Sequential()
    model.add(Dense(units=256, input_shape=(x_shape[1], ), activation='linear'))
    model.add(Dense(units=7,activation='linear'))
    model.summary()
    return model

In [428]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

In [429]:
LinearModel = bulidModel(np_train_X.shape)
LinearModel.compile(loss=root_mean_squared_error, optimizer='rmsprop')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_53 (Dense)             (None, 256)               11008     
_________________________________________________________________
dense_54 (Dense)             (None, 7)                 1799      
Total params: 12,807
Trainable params: 12,807
Non-trainable params: 0
_________________________________________________________________


In [430]:
train_history = LinearModel.fit(np_train_X , np_train_Y , batch_size=64 , epochs=1000 , verbose=1, validation_split=0.2)

Train on 41 samples, validate on 11 samples
Epoch 1/1000
41/41 [==============================] - 1s 20ms/step - loss: 30567.2266 - val_loss: 14461.4521
Epoch 2/1000
41/41 [==============================] - 0s 112us/step - loss: 15776.6025 - val_loss: 4429.2778
Epoch 3/1000
41/41 [==============================] - 0s 168us/step - loss: 5393.2559 - val_loss: 4310.5698
Epoch 4/1000
41/41 [==============================] - 0s 193us/step - loss: 4305.2861 - val_loss: 5001.7690
Epoch 5/1000
41/41 [==============================] - 0s 198us/step - loss: 5687.5220 - val_loss: 4928.0449
Epoch 6/1000
41/41 [==============================] - 0s 309us/step - loss: 5402.0488 - val_loss: 5139.2954
Epoch 7/1000
41/41 [==============================] - 0s 207us/step - loss: 5771.1191 - val_loss: 4384.3120
Epoch 8/1000
41/41 [==============================] - 0s 228us/step - loss: 4644.1704 - val_loss: 4109.6411
Epoch 9/1000
41/41 [==============================] - 0s 228us/step - loss: 4883.4805 - va

Epoch 76/1000
41/41 [==============================] - 0s 151us/step - loss: 3489.7070 - val_loss: 2967.7678
Epoch 77/1000
41/41 [==============================] - 0s 147us/step - loss: 3561.0752 - val_loss: 3262.9717
Epoch 78/1000
41/41 [==============================] - 0s 158us/step - loss: 3702.9890 - val_loss: 3101.5164
Epoch 79/1000
41/41 [==============================] - 0s 167us/step - loss: 3529.5222 - val_loss: 2917.7866
Epoch 80/1000
41/41 [==============================] - 0s 145us/step - loss: 3509.1956 - val_loss: 3165.0674
Epoch 81/1000
41/41 [==============================] - 0s 104us/step - loss: 3484.2332 - val_loss: 2736.5876
Epoch 82/1000
41/41 [==============================] - 0s 111us/step - loss: 3435.2222 - val_loss: 3183.5735
Epoch 83/1000
41/41 [==============================] - 0s 203us/step - loss: 3430.8479 - val_loss: 2568.6587
Epoch 84/1000
41/41 [==============================] - 0s 133us/step - loss: 3317.8660 - val_loss: 3114.4094
Epoch 85/1000
41/41

Epoch 151/1000
41/41 [==============================] - 0s 141us/step - loss: 3466.3408 - val_loss: 2764.3350
Epoch 152/1000
41/41 [==============================] - 0s 163us/step - loss: 3355.2314 - val_loss: 3127.9639
Epoch 153/1000
41/41 [==============================] - 0s 131us/step - loss: 3327.3042 - val_loss: 2466.8291
Epoch 154/1000
41/41 [==============================] - 0s 142us/step - loss: 3241.7283 - val_loss: 3231.0977
Epoch 155/1000
41/41 [==============================] - 0s 125us/step - loss: 3249.8347 - val_loss: 2238.7278
Epoch 156/1000
41/41 [==============================] - 0s 137us/step - loss: 3222.0847 - val_loss: 3305.8162
Epoch 157/1000
41/41 [==============================] - 0s 163us/step - loss: 3200.1501 - val_loss: 2190.9841
Epoch 158/1000
41/41 [==============================] - 0s 113us/step - loss: 3204.3838 - val_loss: 3301.6858
Epoch 159/1000
41/41 [==============================] - 0s 151us/step - loss: 3254.4058 - val_loss: 2546.2344
Epoch 160/

Epoch 226/1000
41/41 [==============================] - 0s 123us/step - loss: 3192.6416 - val_loss: 2726.9846
Epoch 227/1000
41/41 [==============================] - 0s 150us/step - loss: 3345.9653 - val_loss: 2932.3281
Epoch 228/1000
41/41 [==============================] - 0s 123us/step - loss: 3093.3018 - val_loss: 2492.7366
Epoch 229/1000
41/41 [==============================] - 0s 110us/step - loss: 3124.5581 - val_loss: 2839.2329
Epoch 230/1000
41/41 [==============================] - 0s 152us/step - loss: 2962.0815 - val_loss: 2285.3948
Epoch 231/1000
41/41 [==============================] - 0s 114us/step - loss: 2991.0723 - val_loss: 2937.2634
Epoch 232/1000
41/41 [==============================] - 0s 93us/step - loss: 2948.7825 - val_loss: 2266.2803
Epoch 233/1000
41/41 [==============================] - 0s 163us/step - loss: 3105.0923 - val_loss: 3301.6882
Epoch 234/1000
41/41 [==============================] - 0s 116us/step - loss: 3193.6030 - val_loss: 2526.2629
Epoch 235/1

Epoch 301/1000
41/41 [==============================] - 0s 143us/step - loss: 3156.3877 - val_loss: 3029.1416
Epoch 302/1000
41/41 [==============================] - 0s 139us/step - loss: 3048.8730 - val_loss: 2626.8123
Epoch 303/1000
41/41 [==============================] - 0s 148us/step - loss: 3215.3933 - val_loss: 2784.5081
Epoch 304/1000
41/41 [==============================] - 0s 142us/step - loss: 3024.7322 - val_loss: 2707.1533
Epoch 305/1000
41/41 [==============================] - 0s 116us/step - loss: 3046.4866 - val_loss: 2544.9873
Epoch 306/1000
41/41 [==============================] - 0s 128us/step - loss: 2982.3381 - val_loss: 2817.0720
Epoch 307/1000
41/41 [==============================] - 0s 160us/step - loss: 2951.0178 - val_loss: 2463.4663
Epoch 308/1000
41/41 [==============================] - 0s 106us/step - loss: 3053.8730 - val_loss: 2902.8232
Epoch 309/1000
41/41 [==============================] - 0s 192us/step - loss: 2953.7075 - val_loss: 2425.8816
Epoch 310/

...

Epoch 676/1000
41/41 [==============================] - 0s 168us/step - loss: 2725.3562 - val_loss: 2685.6912
Epoch 677/1000
41/41 [==============================] - 0s 165us/step - loss: 2548.5623 - val_loss: 2093.9971
Epoch 678/1000
41/41 [==============================] - 0s 156us/step - loss: 2690.2373 - val_loss: 2577.0830
Epoch 679/1000
41/41 [==============================] - 0s 198us/step - loss: 2392.4448 - val_loss: 1921.7539
Epoch 680/1000
41/41 [==============================] - 0s 180us/step - loss: 2556.9587 - val_loss: 2575.0186
Epoch 681/1000
41/41 [==============================] - 0s 167us/step - loss: 2344.4753 - val_loss: 1819.1005
Epoch 682/1000
41/41 [==============================] - 0s 187us/step - loss: 2574.3816 - val_loss: 2864.7087
Epoch 683/1000
41/41 [==============================] - 0s 184us/step - loss: 2541.8762 - val_loss: 1821.8066
Epoch 684/1000
41/41 [==============================] - 0s 149us/step - loss: 2726.2957 - val_loss: 2834.3877
Epoch 685/

Epoch 751/1000
41/41 [==============================] - 0s 313us/step - loss: 2437.1436 - val_loss: 2582.3049
Epoch 752/1000
41/41 [==============================] - 0s 214us/step - loss: 2448.7568 - val_loss: 2215.4348
Epoch 753/1000
41/41 [==============================] - 0s 115us/step - loss: 2679.4819 - val_loss: 2377.1069
Epoch 754/1000
41/41 [==============================] - 0s 169us/step - loss: 2498.0222 - val_loss: 2319.6201
Epoch 755/1000
41/41 [==============================] - 0s 120us/step - loss: 2516.6682 - val_loss: 1938.8129
Epoch 756/1000
41/41 [==============================] - 0s 137us/step - loss: 2372.6877 - val_loss: 2552.6453
Epoch 757/1000
41/41 [==============================] - 0s 181us/step - loss: 2430.0093 - val_loss: 1602.2781
Epoch 758/1000
41/41 [==============================] - 0s 166us/step - loss: 2410.7551 - val_loss: 2950.6257
Epoch 759/1000
41/41 [==============================] - 0s 150us/step - loss: 2484.5801 - val_loss: 1716.8837
Epoch 760/

Epoch 826/1000
41/41 [==============================] - 0s 152us/step - loss: 2432.7988 - val_loss: 1912.3915
Epoch 827/1000
41/41 [==============================] - 0s 128us/step - loss: 2573.3369 - val_loss: 2562.9900
Epoch 828/1000
41/41 [==============================] - 0s 152us/step - loss: 2360.7700 - val_loss: 1916.9036
Epoch 829/1000
41/41 [==============================] - 0s 139us/step - loss: 2461.0229 - val_loss: 2416.3174
Epoch 830/1000
41/41 [==============================] - 0s 126us/step - loss: 2289.7676 - val_loss: 1892.3179
Epoch 831/1000
41/41 [==============================] - 0s 148us/step - loss: 2389.1172 - val_loss: 2463.4802
Epoch 832/1000
41/41 [==============================] - 0s 122us/step - loss: 2321.7651 - val_loss: 1856.7269
Epoch 833/1000
41/41 [==============================] - 0s 157us/step - loss: 2405.8308 - val_loss: 2522.4502
Epoch 834/1000
41/41 [==============================] - 0s 159us/step - loss: 2354.9983 - val_loss: 1782.6992
Epoch 835/

Epoch 901/1000
41/41 [==============================] - 0s 132us/step - loss: 2326.2952 - val_loss: 2020.5176
Epoch 902/1000
41/41 [==============================] - 0s 231us/step - loss: 2414.8918 - val_loss: 2329.1311
Epoch 903/1000
41/41 [==============================] - 0s 98us/step - loss: 2364.7512 - val_loss: 1936.1818
Epoch 904/1000
41/41 [==============================] - 0s 221us/step - loss: 2374.3523 - val_loss: 2326.9382
Epoch 905/1000
41/41 [==============================] - 0s 187us/step - loss: 2268.8330 - val_loss: 1728.0227
Epoch 906/1000
41/41 [==============================] - 0s 114us/step - loss: 2276.0300 - val_loss: 2374.6951
Epoch 907/1000
41/41 [==============================] - 0s 174us/step - loss: 2189.8372 - val_loss: 1606.1777
Epoch 908/1000
41/41 [==============================] - 0s 141us/step - loss: 2272.9561 - val_loss: 2514.8948
Epoch 909/1000
41/41 [==============================] - 0s 307us/step - loss: 2207.4980 - val_loss: 1612.7825
Epoch 910/1

Epoch 976/1000
41/41 [==============================] - 0s 121us/step - loss: 2368.7690 - val_loss: 2628.2800
Epoch 977/1000
41/41 [==============================] - 0s 149us/step - loss: 2231.1316 - val_loss: 1460.6742
Epoch 978/1000
41/41 [==============================] - 0s 137us/step - loss: 2364.6042 - val_loss: 2638.5955
Epoch 979/1000
41/41 [==============================] - 0s 97us/step - loss: 2295.5281 - val_loss: 1544.4879
Epoch 980/1000
41/41 [==============================] - 0s 144us/step - loss: 2313.8386 - val_loss: 2482.9963
Epoch 981/1000
41/41 [==============================] - 0s 139us/step - loss: 2252.5598 - val_loss: 1632.1218
Epoch 982/1000
41/41 [==============================] - 0s 113us/step - loss: 2237.7158 - val_loss: 2322.6184
Epoch 983/1000
41/41 [==============================] - 0s 130us/step - loss: 2222.8877 - val_loss: 1731.3188
Epoch 984/1000
41/41 [==============================] - 0s 108us/step - loss: 2180.4944 - val_loss: 2199.5168
Epoch 985/1

In [431]:
def getTestX(tp_2018):
    findDate = 0
    index = 0
    test_list = []
    for d in tp_2018:
        if d[0] == '20180402':
            findDate = 1
            index += 1
            test_list += d[1:]
        if findDate is 1:
            index += 1
            test_list += d[1:]
            #test_list.append(d)
        if index is 7:
            break
    return test_list

In [432]:
testList = getTestX(tp_2018)
test_2019_list = ['28756', '1887', '6.56', '29140', '1933', '6.63', '30093', '1892', '6.29', '29673', '2054', '6.92', '25810', '2155', '8.35', '24466', '2298', '9.39', '23895', '1655', '6.92']
test_X = testList + test_2019_list
np_test_X = np.array(test_X)
print(test_X)
print(len(test_X))

['30941', '1894', '6.52', '30941', '1894', '6.52', '31154', '1887', '6.45', '27716', '1595', '6.11', '26725', '1744', '6.98', '26158', '1708', '6.99', '25609', '1669', '6.97', '28756', '1887', '6.56', '29140', '1933', '6.63', '30093', '1892', '6.29', '29673', '2054', '6.92', '25810', '2155', '8.35', '24466', '2298', '9.39', '23895', '1655', '6.92']
42


In [433]:
#print(np_test_X.shape)
test = np_test_X.reshape(1, 42)
#print(test.shape)
test_Y = LinearModel.predict(test)
print(test_Y[0])

[26857.31  29449.035 27861.482 25866.74  25155.15  26267.422 21640.375]


In [436]:
def saveCSV(test_Y):
    date = 20180402
    d_list = []
    temp_list = []
    index = 0
    for t in test_Y:
        temp_list.append(str(date+index))
        temp_list.append(t)
        d_list.append(temp_list)
        temp_list = []
        index += 1
    df = pd.DataFrame(d_list, columns=['date', 'peak_load(MW)'])
    return df

In [437]:
df = saveCSV(test_Y[0])
df.to_csv('submission.csv', index=0)